### Decoding

In [ ]:
import numpy as np
import cv2
import pandas as pd
import os
from PIL import Image
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('./open/train.csv')
df.head()

In [ ]:
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0::2], s[1::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')

In [ ]:
mask_dir = './mask_images/'
os.makedirs(mask_dir, exist_ok=True)

In [ ]:
for index, row in df.iterrows():
    img_id = row['img_id']
    rle = row['mask_rle']

    mask = rle_decode(rle, shape=(1024, 1024))

    mask_path = os.path.join(mask_dir, f'{img_id}_mask.png')
    cv2.imwrite(mask_path, mask * 255)

### Rotation Image

In [ ]:
def rotate_and_flip(image_path, save_path):
    image = Image.open(image_path)

    # 이미지를 오른쪽으로 90도 회전
    rotated_image = image.rotate(-90, expand=True)

    # 이미지 좌우반전
    flipped_image = rotated_image.transpose(Image.FLIP_LEFT_RIGHT)

    # 원본 이미지의 경로와 파일 이름 분리
    image_dir, image_name = os.path.split(image_path)

    # 변환된 이미지 저장
    flipped_image.save(save_path)

In [ ]:
folder_path = './data/mask_images/'

file_list = os.listdir(folder_path)

image_files = [file for file in file_list if file.endswith(('.jpg', '.jpeg', '.png', '.gif'))]

output_folder_path = './data/rotate_mask_image/'

if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# 각 이미지에 rotate_and_flip 함수 적용 및 저장
for image_file in tqdm(image_files):
    # 이미지 파일의 전체 경로 생성
    image_path = os.path.join(folder_path, image_file)

    # 이미지 파일의 이름 추출
    image_name = os.path.splitext(image_file)[0]

    # 회전 및 반전된 이미지 저장 경로 생성
    save_path = os.path.join(output_folder_path, "rotated_and_flipped_" + image_name + ".jpg")

    # rotate_and_flip 함수 호출
    rotate_and_flip(image_path, save_path)

  0%|          | 0/7140 [00:00<?, ?it/s]

/tmp/ipykernel_273707/2133972066.py:8: DeprecationWarning: FLIP_LEFT_RIGHT is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.FLIP_LEFT_RIGHT instead.
  flipped_image = rotated_image.transpose(Image.FLIP_LEFT_RIGHT)
